In [ ]:
import openai
import os
from dotenv import load_dotenv
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain

import pymysql
import locale
import gradio as gr
import pandas as pd

# .env 파일에서 환경 변수 로드
load_dotenv()

# OpenAI API 설정
openai.api_key = os.getenv("OPENAI_API_KEY")

# MySQL 데이터베이스 연결 함수
def connect_to_db():
    return pymysql.connect(
        host='127.0.0.1',
        port=3306,
        user='root',
        passwd='seul0899',
        db='saramin',
        charset="utf8mb4"
    )

# 데이터 가져오기 함수
def fetch_from_saramin_refined():
    connection = connect_to_db()
    cursor = connection.cursor(pymysql.cursors.DictCursor)
    cursor.execute("SELECT * FROM saramin_refined")
    fetch_result = cursor.fetchall()
    connection.close()
    return fetch_result

# 문서 분할 함수
def chunk_document(doc_text, chunk_size=1000, chunk_overlap=100):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return text_splitter.split_text(doc_text)

# 데이터를 문서로 변환
def convert_to_documents(courses):
    documents = []
    for course in courses:
        doc_text = "\n".join([f"{key}: {value}" for key, value in course.items()])
        chunks = chunk_document(doc_text)
        documents.extend([Document(page_content=chunk) for chunk in chunks])
    return documents

# 벡터 스토어 생성
def create_vector_store(documents):
    embeddings = OpenAIEmbeddings()
    return FAISS.from_documents(documents, embeddings)

# 챗봇 생성 (수정됨)
def create_chatbot(vector_store):
    chat_model = ChatOpenAI(model="gpt-4o", temperature=0.2)

    # 프롬프트 템플릿 수정 (시스템 & 사용자 메시지 포함)
    prompt_template = PromptTemplate(
        input_variables=["question", "context"],
        template="""
        시스템 프롬프트
        1. 당신은 잡Job의 채용 전문가입니다. 
        2. 사용자의 요구에 맞는 채용 정보를 정확히 제공하세요. 
        3. 제공 정보에는 해당 직무의 '공고제목','공고링크', '기업명' KEY의 VALUE을 반드시 포함하며 이 정보는 수정없이 원본 정보 그대로 전달합니다. 
        4. 'refined_content' key의 value 중 [채용 공고 주요 내용]을 우선적으로 참고하여 답변합니다. [채용 공고 내용]은 사용자의 추가 질문이 있을경우 참고하여 답변하세요
        5. 채용 정보와 관련없는 질문에는 주어진 데이터가 아닌 스스로 답변하되, 자연스럽게 채용 공고와 관련된 질문을 하도록 유도 하세요
        
        질문: {question}
        
        관련 문서:
        {context}
        """
    )

    llm_chain = LLMChain(llm=chat_model, prompt=prompt_template)  # LLMChain에 프롬프트 적용

    return ConversationalRetrievalChain.from_llm(
        llm=chat_model,
        retriever=vector_store.as_retriever(),
        return_source_documents=True,
        verbose=True,
        combine_docs_chain_kwargs={"prompt": prompt_template}  # ✅ 올바르게 수정됨
    )


# 답변 생성 함수 (수정됨)
def get_answer(question, retrieval_chain, chat_history):
    chat_history.append((question, ""))
    
    # 검색 결과가 있는 경우만 문서 활용
    result = retrieval_chain.invoke({"question": question, "chat_history": chat_history})
    
    # 검색된 문서가 없으면 LLM 단독 응답
    if not result.get("source_documents"):
        llm = ChatOpenAI(model="gpt-4o", temperature=0.2)
        standalone_response = llm.invoke(question)  # LLM만 사용
        return standalone_response, []

    return result.get("answer", ""), result.get("source_documents", [])


# 메인 실행 함수
def main():
    locale.setlocale(locale.LC_ALL, 'ko_KR.UTF-8')
    courses = fetch_from_saramin_refined()
    documents = convert_to_documents(courses)
    vector_store = create_vector_store(documents)
    
    retrieval_chain = create_chatbot(vector_store)
    chat_history = []
    
    def chat_interface(user_input):
        answer, _ = get_answer(user_input, retrieval_chain, chat_history)
        return [(user_input, answer)]
    
    with gr.Blocks() as demo:
        gr.Markdown("<h1 style='text-align: center; color: #6c2dc7;'>잡Job</h1>")
        gr.Image("C:/Users/seul/Downloads/luke-chesser-eICUFSeirc0-unsplash.jpg")
        chatbot = gr.Chatbot()
        user_input = gr.Textbox(label="질문을 입력하세요", placeholder="질문을 입력하세요")
        user_input.submit(chat_interface, inputs=user_input, outputs=chatbot)
    
    demo.launch()

if __name__ == "__main__":
    main()


c:\Users\seul\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-hGK6LnUe-py3.11\Lib\site-packages\gradio\components\chatbot.py:284: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7882

To create a public link, set `share=True` in `launch()`.


In [ ]:
# 얘가 '영업'이면 보통 어떤 직무를 말하는지 잘 모르는거 같다